# **Feature Extraction Methods: Imbalanced Data Without Annotations**

- *Key Features*: [MFCCs, Mel-Spectrograms, Chroma Frequencies, CQT]
- *Key Manipulations*: [Varying Window Sizes, Normalization, Average Pooling (Compression)]
- *Process Assistence*: [Converting them to numpy arrays now, easy label access across features]
- *Conversion*: [To numpy arrays and pkl files]


In [1]:
# Standard libraries
import numpy as np
import pandas as pd
import os
import time

# Libraries for audio
from IPython.display import Audio
import librosa
import librosa.display

# Training and Testing Split
from sklearn.model_selection import train_test_split

# for normalization & avgpooling features
import tensorflow as tf
# for preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import resample

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
import IPython.display as ipd
from functools import partial

In [4]:
# Variabels to be reused
path = 'C:/Users/thato/Documents/Final-Year-Project/Dataset/Final-Version-Of-Bird-Classification-Project/audio_files' 
npy_path = 'C:/Users/thato/Documents/Final-Year-Project/Dataset/Final-Version-Of-Bird-Classification-Project/train_audio_npy/' 
train_csv = 'C:/Users/thato/Documents/Final-Year-Project/Dataset/Final-Version-Of-Bird-Classification-Project/train-not-annotated.csv' 
annotated_train_csv = 'C:/Users/thato/Documents/Final-Year-Project/Dataset/Final-Version-Of-Bird-Classification-Project/train-annotated.csv'
not_annotated_splt = 'C:/Users/thato/Documents/Final-Year-Project/Dataset/Final-Version-Of-Bird-Classification-Project/trainval-split/trainval.csv'
sr = 22050

In [5]:
trainval_data = pd.read_csv(not_annotated_splt)
train_data = trainval_data[trainval_data['set'] == 'tr']
val_data = trainval_data[trainval_data['set'] == 'val']

In [7]:
train_data['species'].value_counts()

species
Himantopus himantopus        43
Alcedo atthis                37
Tachybaptus ruficollis       36
Botaurus stellaris           32
Gallinula chloropus          30
Charadrius alexandrinus      30
Porphyrio porphyrio          29
Fulica atra                  28
Acrocephalus melanopogon     28
Ardea purpurea               24
Motacilla flava              20
Acrocephalus scirpaceus      20
Circus aeruginosus           20
Acrocephalus arundinaceus    19
Ixobrychus minutus           19
Ciconia ciconia              19
Dendrocopos minor            19
Coracias garrulus            13
Anas platyrhynchos           12
Anas strepera                 4
Name: count, dtype: int64

# **Creating a class to do the extraction**

In [12]:
class Extraction:
  def __init__(self, train_df, val_df, window_size, overlap=0.5, npy_path=npy_path, sr=sr, n_mels=128, n_mfcc=20, n_chroma=12, n_cqt=84, hoplength=256, features=['mfcc'], normalize=True, avgpool=False, augment=True):
    """
    Instantiate the Extraction class to extract features.

    Parameters:
      sr (int): Sample rate of the audio files.
      n_mfccs (int): Number of MFCCs to extract.
      n_mels (int): Number of Mel bands to extract.
      n_chroma (int): Number of chroma bins to use.
      n_cqt (int): Number of CQT bins to use.
      features (list): List of features to extract.
        accepted features: 'mfcc', 'chroma', 'cqt', 'melspectrogram'.
      normalize (bool): Whether to normalize the features.
      avgpool (bool): Whether to avgpool the features.
    """
    
    self.train_df = train_df
    self.val_df = val_df
    self.npy_path = npy_path
    self.window_size = window_size
    self.overlap = overlap
    self.sr = sr
    self.n_mels = n_mels
    self.n_mfcc = n_mfcc
    self.n_chroma = n_chroma
    self.n_cqt = n_cqt
    self.hoplength = hoplength

     # confirm features have been specified
    assert len(features) != 0, "Must Specify At Least One Feature In The Form Of A List."
    self.features = features

    self.accepted_feature = ['mfcc', 'chroma', 'cqt', 'melspectrogram']
    for feature in self.features:
      assert feature in self.accepted_feature, f"{feature} is not an accepted feature, only 'mfcc', 'chroma', 'cqt', 'melspectrogram' are accepted features."

    self.normalize = normalize
    self.avgpool = avgpool
    self.augment = augment

    print(f"Train DataFrame shape: {train_df.shape}")
    print(f"Validation DataFrame shape: {val_df.shape}")

    # extract train and val labels and features
    self.train_y, self.train_features, self.train_ids = self.feature_extraction(self.train_df, window_size=self.window_size)
    self.val_y, self.val_features, self.val_ids = self.feature_extraction(self.val_df, window_size=self.window_size, augment=False)

    # process the features by average pooling
    self.train_features_2D, self.val_features_2D, self.train_features_1D, self.val_features_1D = self.process_features(self.train_features, self.val_features)

  def random_augmentation(self, audio):
        """
        Apply random augmentation to the audio: pitch shift, time stretch, or add noise.
        """
        aug_type = np.random.choice(['pitch_up', 'pitch_down', 'noise', 'stretch', 'shrink'])

        if aug_type == 'pitch_up':
            # Random pitch up (between 1 and 4 semitones)
            pitch_shift = np.random.uniform(1, 4)
            audio = librosa.effects.pitch_shift(audio, sr=self.sr, n_steps=pitch_shift)
            return audio
        elif aug_type == 'pitch_down':
            # Random pitch down (between -1 and -4 semitones)
            pitch_shift = np.random.uniform(-4, -1)
            audio = librosa.effects.pitch_shift(audio, sr=self.sr, n_steps=pitch_shift)
            return audio
        elif aug_type == 'noise':
            # Add random noise at a fraction of the original audio volume
            noise = self.generate_pink_noise(len(audio))
            audio = audio + 0.01 * np.random.uniform(0.2, 0.5) * np.max(audio) * noise
            return audio
        elif aug_type == 'stretch':
            # Randomly stretch audio (speed up)
            stretch_factor = np.random.uniform(1.1, 1.5)
            audio = librosa.effects.time_stretch(audio, rate=stretch_factor)
            return audio
        elif aug_type == 'shrink':
            # Randomly shrink audio (slow down)
            stretch_factor = np.random.uniform(0.8, 0.9)
            audio = librosa.effects.time_stretch(audio, rate=stretch_factor)
            return audio
        

  def normalize_audio(self, audio):
    return (audio - np.min(audio)) / (np.max(audio) - np.min(audio))
  
  def generate_pink_noise(self, num_samples):
    white_noise = np.random.randn(num_samples)
    
    # Apply a filter to convert white noise into pink noise (1/f noise)
    X = np.fft.rfft(white_noise)
    S = np.arange(1, len(X) + 1)  # Frequency scaling
    pink_noise = np.fft.irfft(X / S)

    if len(pink_noise) < num_samples:
        # Pad with zeros if the length is less than num_samples
        pink_noise = np.pad(pink_noise, (0, num_samples - len(pink_noise)), mode='constant')
    elif len(pink_noise) > num_samples:
        # Trim if necessary
        pink_noise = pink_noise[:num_samples]
    
    return self.normalize_audio(pink_noise)
  
  def pad_with_noise(self, audio_data, window_length, window_samples):
    current_length = librosa.get_duration(y=audio_data, sr=self.sr)

    if current_length > window_length:
        return audio_data
    
    target_length_samples = int(window_length * sr) 
    current_length_samples = window_samples
    padding_length_samples = target_length_samples - current_length_samples

    assert target_length_samples == (current_length_samples+padding_length_samples)
    
    # Generate pink noise to pad with
    pink_noise = self.generate_pink_noise(padding_length_samples)
    padded_audio = np.concatenate([audio_data, pink_noise])
    # if len(padded_audio) < target_length_samples:
    #     padded_audio = np.append(padded_audio, self.generate_pink_noise(1))

    assert target_length_samples == len(padded_audio)
    
    return padded_audio

#-------------------------Feature Extraction---------------------------------------
  def extract_mfcc(self, window):
    mfcc = librosa.feature.mfcc(y=window, sr=self.sr, n_mfcc=self.n_mfcc, hop_length=self.hoplength)
    if self.normalize:
      return librosa.util.normalize(mfcc)
    else:
      return mfcc


  def extract_chroma(self, window):
    chroma = librosa.feature.chroma_stft(y=window, sr=self.sr, n_chroma=self.n_chroma, hop_length=self.hoplength)
    if self.normalize:
      return librosa.util.normalize(chroma)
    else:
      return chroma
   

  def extract_cqt(self, window):
    cqt = librosa.cqt(y=window, sr=sr, hop_length=self.hoplength, n_bins=self.n_cqt)
    cqt_db = librosa.amplitude_to_db(np.abs(cqt), ref=np.max)
    return cqt_db

  def extract_melspectrogram(self, window):
    mel = librosa.feature.melspectrogram(y=window, sr=self.sr, n_mels=self.n_mels, hop_length=self.hoplength)
    mel_db = librosa.power_to_db(mel, ref=np.max)
    if self.normalize:
      return librosa.util.normalize(mel_db)
    else:
      return mel_db
    
  def avgpooling(self, train_X, val_X, n_time, n_features):
    """
    Average pooling the train and val features.

    Parameters:
      train_X (npy): Training feature array of shape (batch_size, n_features, n_time)
      val_X (npy): Validation feature array of shape (batch_size, n_features, n_time)
      n_time (int): Time axis
      n_features (int): Feature axis

    Returns:
      train_X (npy): Avgpooled training feature array of shape (batch_size, n_features)
      val_X (npy): Avgpooled validation feature array of shape (batch_size, n_features)
    """
    # Clear the Keras session
    tf.keras.backend.clear_session()
    
    # Create the Keras input layer with shape (n_features, n_time)
    input_layer = tf.keras.layers.Input(shape=(n_features, n_time))
    
    # Apply average pooling over the time axis (axis=-1) to reduce n_time
    avg_pool = tf.keras.layers.Lambda(lambda x: tf.reduce_mean(x, axis=-1))(input_layer)
    
    # Build the model
    pooling_model = tf.keras.models.Model(inputs=input_layer, outputs=avg_pool)

    # Use the model to apply average pooling on the training and validation features
    train_X = pooling_model.predict(train_X)
    val_X = pooling_model.predict(val_X)

    return train_X, val_X

    
  def process_features(self, train_features_dict, val_features_dict):
    train_copy = train_features_dict.copy()
    val_copy = val_features_dict.copy()
    for each in train_copy.keys():
      
      if each == 'mfcc':
        n_features=self.n_mfcc
      elif each == 'chroma':
        n_features=self.n_chroma
      elif each == 'cqt':
        n_features=self.n_cqt
      elif each == 'melspectrogram':
        n_features=self.n_mels
      
      train_feature = train_copy[each]
      val_feature = val_copy[each]

      if self.avgpool:
        train_copy[each], val_copy[each] = self.avgpooling(train_feature, val_feature, n_time=train_feature.shape[2], n_features=n_features)
      else:
        train_copy[each], val_copy[each] = train_copy[each], val_copy[each]
    
    return train_features_dict, val_features_dict, train_copy, val_copy
      

  def feature_extraction(self, dataframe, window_size, augment=True):
    y = [] # To hold the labels
    ids = [] # To hold the audio name (going to use it to aggregate the results for one audio file)
    features_dict = {item: [] for item in self.features} # Create a key for each feature listed
    print(f"Number of rows in dataframe: {len(dataframe)}")
    for _, row in tqdm(dataframe.iterrows(), desc="Processing data", total=len(dataframe)):
          label = row['species']
          file_path = os.path.join(self.npy_path, row['filename_npy'])
          id = row['audio_name']

          try:
              audio = np.load(file_path)
          except FileNotFoundError:
              print(f"File not found: {file_path}")
              continue


          audio = self.normalize_audio(audio)
          if augment:
                audio = self.random_augmentation(audio)
          audio = self.pad_with_noise(audio, window_length=self.window_size, window_samples=len(audio))
          # print(len(sample))

          window_samples = int(window_size * self.sr)
          hop_samples = int(window_samples * (1 - self.overlap))  # For overlapping

          # Break the audio into windows with the specified overlap
          audio_windows = librosa.util.frame(audio, frame_length=window_samples, hop_length=hop_samples).T
          
          
          # display(label)
          
          for _, window in enumerate(audio_windows):
              
              y.append(label)
              ids.append(id)

              if len(window) < window_samples:
                  if len(window) < 512*2:
                     continue
                  else:
                      window = self.pad_with_noise(window, window_length=window_size)
              
              # Feature Extraction FR --------------------------------------------------------------------
              # dynatically call the extract_x function to extract the listed features
              for feature in self.features:
                extract = f"extract_{feature}"
                if hasattr(self, extract) and callable(func := getattr(self, extract)):
                  features_dict[feature].append(func(window))

          # cast lists to np arrays
    for each in features_dict.keys():
              features_dict[each] = np.array(features_dict[each])

    y = np.array(y)
    ids = np.array(ids)

          # If not using average pooling, return resized features
    return y, features_dict, ids

# **Trying to balance the training set**

In [13]:
# Step 1: Identify the majority class count
species_counts = train_data['species'].value_counts()
majority_class_count = species_counts.max()

# Step 2: Define the target for oversampling (0.7 times the majority class count)
target_count = int(majority_class_count * 0.7)

# Step 3: Separate majority and minority classes
df_majority = train_data[train_data['species'].map(species_counts) == majority_class_count]

# Step 4: Oversample the minority classes to the target count
oversampled_data = pd.DataFrame()

for species, count in species_counts.items():
    species_df = train_data[train_data['species'] == species]
    if count < target_count:
        species_upsampled = resample(species_df, 
                                     replace=True,     # Allow oversampling
                                     n_samples=target_count,  # Target count for each class
                                     random_state=42)  # For reproducibility
        oversampled_data = pd.concat([oversampled_data, species_upsampled])
    else:
        oversampled_data = pd.concat([oversampled_data, species_df])

# Combine with majority class data
train_data_balanced = pd.concat([oversampled_data])

# Check the new class distribution
print(train_data_balanced['species'].value_counts())

species
Himantopus himantopus        43
Alcedo atthis                37
Tachybaptus ruficollis       36
Botaurus stellaris           32
Circus aeruginosus           30
Anas platyrhynchos           30
Coracias garrulus            30
Dendrocopos minor            30
Ciconia ciconia              30
Ixobrychus minutus           30
Acrocephalus arundinaceus    30
Motacilla flava              30
Acrocephalus scirpaceus      30
Ardea purpurea               30
Acrocephalus melanopogon     30
Fulica atra                  30
Porphyrio porphyrio          30
Charadrius alexandrinus      30
Gallinula chloropus          30
Anas strepera                30
Name: count, dtype: int64


# **Average Pooling**

## **Window Size = 6s**

### **['melspectrogram', 'mfcc', 'chroma', 'cqt']**

In [14]:
features_list = ['melspectrogram', 'mfcc', 'chroma', 'cqt']

In [15]:
features = Extraction(train_data_balanced,
                      val_data,
                      window_size=6,
                      features=features_list,
                      avgpool=True
                      )

Train DataFrame shape: (628, 6)
Validation DataFrame shape: (133, 6)
Number of rows in dataframe: 628


Processing data:   0%|          | 0/628 [00:00<?, ?it/s]c:\Users\thato\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
Processing data: 100%|██████████| 628/628 [38:42<00:00,  3.70s/it]  


Number of rows in dataframe: 133


Processing data: 100%|██████████| 133/133 [02:46<00:00,  1.25s/it]



307/307 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
307/307 ━━━━━━━━━━━━━━━━━━━━ 0s 541us/step
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 560us/step
307/307 ━━━━━━━━━━━━━━━━━━━━ 0s 501us/step
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 479us/step
307/307 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 958us/step


In [16]:
train_y = features.train_y
val_y = features.val_y

display(train_y.shape)
display(val_y.shape)

(9800,)

(2028,)

In [17]:
# Avgpooled Features
train_features_1D = features.train_features_1D
for key in train_features_1D.keys():
  display(key)
  display(train_features_1D[key].shape)

'melspectrogram'

(9800, 128)

'mfcc'

(9800, 20)

'chroma'

(9800, 12)

'cqt'

(9800, 84)

In [18]:
# Avgpooled Features
val_features_1D = features.val_features_1D
for key in val_features_1D.keys():
  display(key)
  display(val_features_1D[key].shape)

'melspectrogram'

(2028, 128)

'mfcc'

(2028, 20)

'chroma'

(2028, 12)

'cqt'

(2028, 84)

In [19]:
# Not avgpooled Features
train_features_2D = features.train_features_2D
for key in train_features_2D.keys():
  display(key)
  display(train_features_2D[key].shape)

'melspectrogram'

(9800, 128, 517)

'mfcc'

(9800, 20, 517)

'chroma'

(9800, 12, 517)

'cqt'

(9800, 84, 517)

In [20]:
# Not avgpooled Features
val_features_2D = features.val_features_2D
for key in val_features_2D.keys():
  display(key)
  display(val_features_2D[key].shape)

'melspectrogram'

(2028, 128, 517)

'mfcc'

(2028, 20, 517)

'chroma'

(2028, 12, 517)

'cqt'

(2028, 84, 517)

In [21]:
train_ids = features.train_ids
val_ids = features.val_ids

### Encode Classes

In [22]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [15 15 15 ...  5  5  5]
Encoded validation labels: [15 15 15 ... 12 12 12]


In [23]:
# Avg Pooled

train_features_1D['label'] = train_y_encoded
val_features_1D['label'] = val_y_encoded

train_features_1D['id'] = train_ids
val_features_1D['id'] = val_ids

# Not Avg Pooled

train_features_2D['label'] = train_y_encoded
val_features_2D['label'] = val_y_encoded

train_features_2D['id'] = train_ids
val_features_2D['id'] = val_ids

In [24]:
merged_dict_1D = {'train': train_features_1D, 'val': val_features_1D}
merged_dict_1D

{'train': {'melspectrogram': array([[-0.01737999, -0.9887568 , -0.9906883 , ..., -0.86174977,
          -0.8951953 , -0.9989192 ],
         [-0.01776686, -0.9898095 , -0.9906936 , ..., -0.86467505,
          -0.8967181 , -0.9992211 ],
         [-0.01776693, -0.98981524, -0.99070024, ..., -0.869458  ,
          -0.9015527 , -0.9991495 ],
         ...,
         [-0.01774816, -0.87505704, -0.83074677, ..., -0.9999875 ,
          -0.99999493, -1.        ],
         [-0.01776952, -0.8752005 , -0.83170116, ..., -0.9999876 ,
          -0.99999493, -1.        ],
         [-0.01775181, -0.8790601 , -0.83794415, ..., -0.99998736,
          -0.99999493, -1.        ]], dtype=float32),
  'mfcc': array([[-1.        , -0.06654957, -0.12421619, ...,  0.02332132,
           0.01864451,  0.04465649],
         [-1.        , -0.07432997, -0.16210054, ...,  0.02310089,
           0.01966888,  0.04817654],
         [-1.        , -0.07249828, -0.15793009, ...,  0.02668171,
           0.01590823,  0.04401247]

In [25]:
merged_dict_2D = {'train': train_features_2D, 'val': val_features_2D}
merged_dict_2D

{'train': {'melspectrogram': array([[[-1.75276976e-02,  0.00000000e+00, -7.71940732e-03, ...,
           -1.61800776e-02, -4.91444208e-03, -2.97452265e-04],
          [-1.94064215e-01, -2.12444767e-01, -2.69574255e-01, ...,
           -3.59871984e-01, -2.52936304e-01, -2.02870801e-01],
          [-2.68468022e-01, -2.86741257e-01, -3.42513889e-01, ...,
           -4.10629153e-01, -3.25488627e-01, -2.76673973e-01],
          ...,
          [-9.98166680e-01, -9.98050034e-01, -9.95472729e-01, ...,
           -9.54798877e-01, -9.99729514e-01, -1.00000000e+00],
          [-9.99085188e-01, -9.98989701e-01, -9.97560978e-01, ...,
           -9.73816872e-01, -9.94321585e-01, -9.91682649e-01],
          [-1.00000000e+00, -1.00000000e+00, -1.00000000e+00, ...,
           -1.00000000e+00, -1.00000000e+00, -9.94185746e-01]],
  
         [[-1.73332524e-02,  0.00000000e+00, -8.11586156e-03, ...,
           -1.66160017e-02, -5.43642510e-03, -8.62621004e-04],
          [-1.93515822e-01, -2.12081432e-01,

### Save the merged dictionary to a pkl

In [26]:
# Avrg Pooled
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Final-Version-Of-Bird-Classification-Project/feature-extraction/NotAnnotated/Augmented/AveragePooled/split_features_6s_all_1D.pkl', 'wb') as file:
  pickle.dump(merged_dict_1D, file)
del file

In [27]:
# Not Avg Pooled
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Final-Version-Of-Bird-Classification-Project/feature-extraction/NotAnnotated/Augmented/NotAveragePooled/split_features_6s_all_2D.pkl', 'wb') as file:
  pickle.dump(merged_dict_2D, file)
del file

## **Window Size = 3s**

### **['melspectrogram', 'mfcc', 'chroma', 'cqt']**

In [28]:
features_list = ['melspectrogram', 'mfcc', 'chroma', 'cqt']

In [29]:
features = Extraction(train_data_balanced,
                      val_data,
                      window_size=3,
                      features=features_list,
                      avgpool=True
                      )

Train DataFrame shape: (628, 6)
Validation DataFrame shape: (133, 6)
Number of rows in dataframe: 628


Processing data:   0%|          | 0/628 [00:00<?, ?it/s]c:\Users\thato\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
Processing data: 100%|██████████| 628/628 [52:12<00:00,  4.99s/it]  


Number of rows in dataframe: 133


Processing data: 100%|██████████| 133/133 [03:47<00:00,  1.71s/it]


618/618 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
618/618 ━━━━━━━━━━━━━━━━━━━━ 1s 985us/step
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 437us/step
618/618 ━━━━━━━━━━━━━━━━━━━━ 0s 505us/step
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 400us/step
618/618 ━━━━━━━━━━━━━━━━━━━━ 1s 760us/step
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [30]:
train_y = features.train_y
val_y = features.val_y

display(train_y.shape)
display(val_y.shape)

(19766,)

(4231,)

In [31]:
# Avgpooled Features
train_features_1D = features.train_features_1D
for key in train_features_1D.keys():
  display(key)
  display(train_features_1D[key].shape)

'melspectrogram'

(19766, 128)

'mfcc'

(19766, 20)

'chroma'

(19766, 12)

'cqt'

(19766, 84)

In [32]:
# Avgpooled Features
val_features_1D = features.val_features_1D
for key in val_features_1D.keys():
  display(key)
  display(val_features_1D[key].shape)

'melspectrogram'

(4231, 128)

'mfcc'

(4231, 20)

'chroma'

(4231, 12)

'cqt'

(4231, 84)

In [33]:
# Not avgpooled Features
train_features_2D = features.train_features_2D
for key in train_features_2D.keys():
  display(key)
  display(train_features_2D[key].shape)

'melspectrogram'

(19766, 128, 259)

'mfcc'

(19766, 20, 259)

'chroma'

(19766, 12, 259)

'cqt'

(19766, 84, 259)

In [34]:
# Not avgpooled Features
val_features_2D = features.val_features_2D
for key in val_features_2D.keys():
  display(key)
  display(val_features_2D[key].shape)

'melspectrogram'

(4231, 128, 259)

'mfcc'

(4231, 20, 259)

'chroma'

(4231, 12, 259)

'cqt'

(4231, 84, 259)

In [35]:
train_ids = features.train_ids
val_ids = features.val_ids

### Encode Classes

In [36]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [15 15 15 ...  5  5  5]
Encoded validation labels: [15 15 15 ... 12 12 12]


In [37]:
# Avg Pooled

train_features_1D['label'] = train_y_encoded
val_features_1D['label'] = val_y_encoded

train_features_1D['id'] = train_ids
val_features_1D['id'] = val_ids

# Not Avg Pooled

train_features_2D['label'] = train_y_encoded
val_features_2D['label'] = val_y_encoded

train_features_2D['id'] = train_ids
val_features_2D['id'] = val_ids

In [38]:
merged_dict_1D = {'train': train_features_1D, 'val': val_features_1D}
merged_dict_1D

{'train': {'melspectrogram': array([[-0.03214124, -0.96890205, -0.97514397, ..., -0.8797835 ,
          -0.9103911 , -0.9997485 ],
         [-0.01764496, -0.9780887 , -0.97997576, ..., -0.86167973,
          -0.89395815, -0.99885833],
         [-0.01764421, -0.97806746, -0.9799561 , ..., -0.8564302 ,
          -0.88675773, -0.99834627],
         ...,
         [-0.01767047, -0.85546976, -0.822436  , ..., -0.99997175,
          -0.99998844, -1.        ],
         [-0.01765394, -0.8588135 , -0.8415632 , ..., -0.9999724 ,
          -0.9999887 , -1.        ],
         [-0.01762162, -0.8556395 , -0.8310128 , ..., -0.999972  ,
          -0.99998856, -1.        ]], dtype=float32),
  'mfcc': array([[-1.        , -0.04497185, -0.09596926, ...,  0.03470058,
           0.02980951,  0.03267538],
         [-1.        , -0.07467373, -0.12447219, ...,  0.03321128,
           0.02954703,  0.04010739],
         [-1.        , -0.09055886, -0.12986107, ...,  0.03218639,
           0.03536548,  0.03869862]

In [39]:
merged_dict_2D = {'train': train_features_2D, 'val': val_features_2D}
merged_dict_2D

{'train': {'melspectrogram': array([[[-1.85656399e-02,  0.00000000e+00, -3.36131151e-03, ...,
           -3.11951861e-02, -2.16903072e-02, -2.74765622e-02],
          [-1.81066856e-01, -1.98433727e-01, -2.51744866e-01, ...,
           -3.19155753e-01, -2.45763719e-01, -2.13726968e-01],
          [-2.44508147e-01, -2.61068732e-01, -3.12649548e-01, ...,
           -3.87516648e-01, -3.17697823e-01, -2.86605179e-01],
          ...,
          [-9.15928841e-01, -9.17033315e-01, -9.20726717e-01, ...,
           -9.89644527e-01, -9.96836066e-01, -9.98356640e-01],
          [-9.31999445e-01, -9.32433367e-01, -9.34706330e-01, ...,
           -9.88947153e-01, -9.96383667e-01, -9.97944832e-01],
          [-1.00000000e+00, -1.00000000e+00, -1.00000000e+00, ...,
           -1.00000000e+00, -1.00000000e+00, -1.00000000e+00]],
  
         [[-1.73473042e-02,  0.00000000e+00, -8.11313186e-03, ...,
           -1.30778868e-02, -1.34250743e-03, -6.29630731e-03],
          [-1.93665311e-01, -2.12086484e-01,

### Save the merged dictionary to a pkl

In [40]:
# Avrg Pooled
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Final-Version-Of-Bird-Classification-Project/feature-extraction/NotAnnotated/Augmented/AveragePooled/split_features_3s_all_1D.pkl', 'wb') as file:
  pickle.dump(merged_dict_1D, file)
del file

In [41]:
# Avrg Pooled
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Final-Version-Of-Bird-Classification-Project/feature-extraction/NotAnnotated/Augmented/NotAveragePooled/split_features_3s_all_2D.pkl', 'wb') as file:
  pickle.dump(merged_dict_2D, file)
del file

## **Window Size = 1s**

### **['melspectrogram', 'mfcc', 'chroma', 'cqt']**

In [42]:
features_list = ['melspectrogram', 'mfcc', 'chroma', 'cqt']

In [43]:
features = Extraction(train_data_balanced,
                      val_data,
                      window_size=1,
                      features=features_list,
                      avgpool=True
                      )

Train DataFrame shape: (628, 6)
Validation DataFrame shape: (133, 6)
Number of rows in dataframe: 628


Processing data:   0%|          | 0/628 [00:00<?, ?it/s]

c:\Users\thato\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
Processing data: 100%|██████████| 628/628 [1:28:57<00:00,  8.50s/it]  


Number of rows in dataframe: 133


Processing data: 100%|██████████| 133/133 [10:40<00:00,  4.81s/it]


1978/1978 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step
409/409 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
1978/1978 ━━━━━━━━━━━━━━━━━━━━ 2s 762us/step
409/409 ━━━━━━━━━━━━━━━━━━━━ 0s 442us/step
1978/1978 ━━━━━━━━━━━━━━━━━━━━ 1s 552us/step
409/409 ━━━━━━━━━━━━━━━━━━━━ 0s 420us/step
1978/1978 ━━━━━━━━━━━━━━━━━━━━ 1s 534us/step
409/409 ━━━━━━━━━━━━━━━━━━━━ 0s 533us/step


In [44]:
train_y = features.train_y
val_y = features.val_y

display(train_y.shape)
display(val_y.shape)

(63281,)

(13083,)

In [45]:
# Avgpooled Features
train_features_1D = features.train_features_1D
for key in train_features_1D.keys():
  display(key)
  display(train_features_1D[key].shape)

'melspectrogram'

(63281, 128)

'mfcc'

(63281, 20)

'chroma'

(63281, 12)

'cqt'

(63281, 84)

In [46]:
# Avgpooled Features
val_features_1D = features.val_features_1D
for key in val_features_1D.keys():
  display(key)
  display(val_features_1D[key].shape)

'melspectrogram'

(13083, 128)

'mfcc'

(13083, 20)

'chroma'

(13083, 12)

'cqt'

(13083, 84)

In [47]:
# Not avgpooled Features
train_features_2D = features.train_features_2D
for key in train_features_2D.keys():
  display(key)
  display(train_features_2D[key].shape)

'melspectrogram'

(63281, 128, 87)

'mfcc'

(63281, 20, 87)

'chroma'

(63281, 12, 87)

'cqt'

(63281, 84, 87)

In [48]:
# Not avgpooled Features
val_features_2D = features.val_features_2D
for key in val_features_2D.keys():
  display(key)
  display(val_features_2D[key].shape)

'melspectrogram'

(13083, 128, 87)

'mfcc'

(13083, 20, 87)

'chroma'

(13083, 12, 87)

'cqt'

(13083, 84, 87)

In [49]:
train_ids = features.train_ids
val_ids = features.val_ids

### Encode Classes

In [50]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [15 15 15 ...  5  5  5]
Encoded validation labels: [15 15 15 ... 12 12 12]


In [51]:
# Avg Pooled

train_features_1D['label'] = train_y_encoded
val_features_1D['label'] = val_y_encoded

train_features_1D['id'] = train_ids
val_features_1D['id'] = val_ids

# Not Avg Pooled

train_features_2D['label'] = train_y_encoded
val_features_2D['label'] = val_y_encoded

train_features_2D['id'] = train_ids
val_features_2D['id'] = val_ids

In [52]:
merged_dict_1D = {'train': train_features_1D, 'val': val_features_1D}
merged_dict_1D

{'train': {'melspectrogram': array([[-0.03103661, -0.90614426, -0.9229933 , ..., -0.87592673,
          -0.9008445 , -0.99924344],
         [-0.01720369, -0.9333362 , -0.9391782 , ..., -0.86896664,
          -0.89278305, -0.9972611 ],
         [-0.01720074, -0.9332653 , -0.9390875 , ..., -0.8818467 ,
          -0.90610874, -0.99969536],
         ...,
         [-0.01721118, -0.80984443, -0.799863  , ..., -0.9994762 ,
          -0.99993706, -1.        ],
         [-0.01722834, -0.82910657, -0.8080832 , ..., -0.99950963,
          -0.9999029 , -0.99997866],
         [-0.01722996, -0.8216893 , -0.7997474 , ..., -0.99965185,
          -0.9999266 , -1.        ]], dtype=float32),
  'mfcc': array([[-1.        , -0.0058945 , -0.07317362, ...,  0.04118359,
           0.02143237,  0.01942439],
         [-1.        , -0.02000155, -0.07986975, ...,  0.041522  ,
           0.02601608,  0.0182024 ],
         [-1.        , -0.00311628, -0.07623003, ...,  0.03411018,
           0.02587876,  0.0210204 ]

In [53]:
merged_dict_2D = {'train': train_features_2D, 'val': val_features_2D}
merged_dict_2D

{'train': {'melspectrogram': array([[[-1.81918368e-02,  0.00000000e+00, -3.80866183e-03, ...,
           -2.83528697e-02, -2.04673894e-02, -3.38529013e-02],
          [-1.78424492e-01, -1.95810318e-01, -2.48889446e-01, ...,
           -2.93606758e-01, -2.33252510e-01, -2.10932076e-01],
          [-2.40419298e-01, -2.56736904e-01, -3.07756722e-01, ...,
           -3.65042359e-01, -3.05814385e-01, -2.83785045e-01],
          ...,
          [-9.10081625e-01, -9.10845101e-01, -9.12104130e-01, ...,
           -9.95257854e-01, -9.99354243e-01, -9.99998152e-01],
          [-9.27481830e-01, -9.27993119e-01, -9.30427134e-01, ...,
           -9.94865358e-01, -9.97100115e-01, -9.97618139e-01],
          [-1.00000000e+00, -1.00000000e+00, -1.00000000e+00, ...,
           -1.00000000e+00, -1.00000000e+00, -9.99944270e-01]],
  
         [[-1.73634682e-02,  0.00000000e+00, -8.13149754e-03, ...,
           -9.82963014e-03, -4.79028240e-05, -1.30828759e-02],
          [-1.93872809e-01, -2.12541118e-01,

### Save the merged dictionary to a pkl

In [54]:
# Avrg Pooled
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Final-Version-Of-Bird-Classification-Project/feature-extraction/NotAnnotated/Augmented/AveragePooled/split_features_1s_all_1D.pkl', 'wb') as file:
  pickle.dump(merged_dict_1D, file)
del file

In [55]:
# Avrg Pooled
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Final-Version-Of-Bird-Classification-Project/feature-extraction/NotAnnotated/Augmented/NotAveragePooled/split_features_1s_all_2D.pkl', 'wb') as file:
  pickle.dump(merged_dict_2D, file)
del file

OSError: [Errno 28] No space left on device